In [8]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sklearn
import xgboost as xgb
import warnings
warnings.filterwarnings = ('ignore')

In [3]:
data = pd.read_csv('ajah_deploy.csv')
data.drop('Unnamed: 0', axis = 1, inplace = True)
data.head()

,Location,Price,Bedroom,Bathrooms,Toilet,Location Area,State
0,Ajah,40000000,2,2,3,The Balconies,Lagos
1,Ajah,2000000,3,3,4,Lekki Scheme 2 Off Abraham Adesanya Road,Lagos
2,Ajah,40000000,2,2,3,The Balconies,Lagos
3,Ajah,2000000,3,3,4,Lekki Scheme 2 Off Abraham Adesanya Road,Lagos
4,Ajah,40000000,2,2,3,The Balconies,Lagos


In [12]:
ds = data.drop('State', axis = 1)
df = ds.copy()

In [22]:
df.head()

,Location,Price,Bedroom,Bathrooms,Toilet,Location Area
0,0,2.167433,2,2,3,12
1,0,-0.415806,3,3,4,5
2,0,2.167433,2,2,3,12
3,0,-0.415806,3,3,4,5
4,0,2.167433,2,2,3,12


In [16]:
df.isnull().sum()

Location         0
Price            0
Bedroom          0
Bathrooms        0
Toilet           0
Location Area    0
dtype: int64

In [13]:
df.describe()

,Price,Bedroom,Bathrooms,Toilet
count,7.470000e+02,747.000000,747.000000,747.000000
mean,8.116600e+06,2.009371,1.985274,2.888889
std,1.472007e+07,0.751169,0.751083,0.955987
min,6.500000e+05,1.000000,1.000000,1.000000
25%,8.000000e+05,1.000000,1.000000,2.000000
50%,1.500000e+06,2.000000,2.000000,3.000000
75%,2.000000e+06,3.000000,3.000000,4.000000
max,4.000000e+07,3.000000,3.000000,4.000000


In [14]:
# SCALE THE NUMERICAL COLUMNS
from sklearn.preprocessing import StandardScaler

# Ascertain the numerical columns to scale
cols_to_scale = []
for i in df.select_dtypes(include = 'number').columns:
    if df[i].describe()[2] > 1500:
        cols_to_scale.append(i)

# Scale the necessary numerical columns
import joblib

scaled = {}

for i in cols_to_scale:
    scale = StandardScaler()
    df[i] = scale.fit_transform(df[[i]])
    joblib.dump(scale, open(f'{i}_scaler.pkl', 'wb'))
    scaled[i + '_scaler'] = scale

C:\Users\Fareedah Ajao\AppData\Local\Temp\ipykernel_19508\2958698506.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df[i].describe()[2] > 1500:
C:\Users\Fareedah Ajao\AppData\Local\Temp\ipykernel_19508\2958698506.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df[i].describe()[2] > 1500:
C:\Users\Fareedah Ajao\AppData\Local\Temp\ipykernel_19508\2958698506.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df[i].de

In [15]:
# LABEL ENCODE THE CATEGORICAL COLUMNS
from sklearn.preprocessing import LabelEncoder

encoded = {}  # ................................................................ Have a dictionary to save the encoders
for i in df.select_dtypes(exclude = 'number').columns: # ........................................................ Iterate through the categorical variables
    encode = LabelEncoder() # .................................................. Instantiate the encoder transformer
    df[i] = encode.fit_transform(df[i]) # ...................................... Encode the columns in ech iteration
    joblib.dump(encode, open(f'{i}_encoder.pkl', 'wb')) # ...................... Save the transformer to file for future reusal
    encoded[i +'_encoder'] = encode # .......................................... Save the transformer to the above stated dictionary for easy access

encoded

{'Location_encoder': LabelEncoder(), 'Location Area_encoder': LabelEncoder()}

In [17]:
x = df.drop('Price', axis = 1)
y = df.Price

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.2, random_state = 56)

In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

rf_model = RandomForestRegressor()

rf_model.fit(xtrain, ytrain)

# Model Evaluation
cross_validation = rf_model.predict(xtrain) # Predict the training data
score = r2_score(cross_validation, ytrain).round(2) # Check the accuracy of the model

print(f'The Model is {(r2_score(cross_validation, ytrain).round(2)) * 100}% accurate')
if score > 0.75:
    print('Model did not underfit')
else:
    print('Model is underfitted')

The Model is 100.0% accurate
Model did not underfit


In [20]:
pred = rf_model.predict(xtest)
score = r2_score(pred, ytest).round(2) # Check the accuracy of the model

print(f'The Model is {score * 100}% accurate')
if score > 0.75:
    print('Model did not Overfit')
else:
    print('Model is Overfitted')

The Model is 100.0% accurate
Model did not Overfit


In [21]:
joblib.dump(rf_model, open('HousePriceModel.pkl', 'wb'))

In [23]:
df.columns

Index(['Location', 'Price', 'Bedroom', 'Bathrooms', 'Toilet', 'Location Area'], dtype='object')